In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import json

In [2]:
browser = Browser('chrome')

In [3]:
# Visit the Mars news site
url = 'https://www.billboard.com/charts/tiktok-billboard-top-50/'
browser.visit(url)

In [4]:
# Create a Beautiful Soup object
html = browser.html
soup = soup(html, 'html.parser')

In [5]:
# Extract all the text elements
Top_chart_elements = soup.find_all("div", class_="o-chart-results-list-row-container")

In [6]:
# Create an empty list to store the dictionaries
top_chart_list = []

In [7]:
# Loop through the text elements
for article in Top_chart_elements:
    
    date = "2024-06-08"
    title = article.find("h3", class_="c-title").get_text()
    try:
        artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet").get_text()
    except:
        artist = artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only").get_text()

    
    track_name = title.strip()
    artist_name = artist.strip()
    
    # Store in a dictionary
    article_dict = {
        "title": track_name,
        "artist": artist_name,
        "date": date
    }
    
    # Append the dictionary to the list
    top_chart_list.append(article_dict)

In [25]:
top_chart_list
top_chart_df = pd.DataFrame(top_chart_list)
top_chart_df["URI"] = ""
top_chart_df

,title,artist,date,URI
0,Million Dollar Baby,Tommy Richman,2024-06-08,
1,Tell Ur Girlfriend,Lay Bankz,2024-06-08,
2,U My Everything,Sexyy Red & Drake,2024-06-08,
3,Birds Of A Feather,Billie Eilish,2024-06-08,
4,Not Like Us,Kendrick Lamar,2024-06-08,
5,Crazy,Stephen Kramer Glickman Featuring Marza Wilks,2024-06-08,
6,Type Shit,"Future, Metro Boomin, Travis Scott & Playboi C...",2024-06-08,
7,BAND4BAND,Central Cee & Lil Baby,2024-06-08,
8,Si No Quieres No,Luis R Conriquez x Neton Vega,2024-06-08,
9,Gata Only,FloyyMenor X Cris Mj,2024-06-08,


In [9]:
browser.quit()

In [11]:
import requests

CLIENT_ID = "your_id_here"
CLIENT_SECRET = "your_secret_here"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': "68eea273ba0b4954a6237107625186fe",
    'client_secret': "18597e9724e945c8a64f86a2093aaea3",
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [12]:
artist = "Tommy Richman"
trackName = "Million Dollar Baby"

In [22]:
BASE_URL = 'https://api.spotify.com/v1/'
r = requests.get(BASE_URL + f'search?q={artist} {trackName}&type=track&offset=0&limit=1', headers=headers)
r = r.json()


In [23]:
r

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Tommy+Richman+Million+Dollar+Baby&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1WaFQSHVGZQJTbf0BdxdNo'},
       'href': 'https://api.spotify.com/v1/artists/1WaFQSHVGZQJTbf0BdxdNo',
       'id': '1WaFQSHVGZQJTbf0BdxdNo',
       'name': 'Tommy Richman',
       'type': 'artist',
       'uri': 'spotify:artist:1WaFQSHVGZQJTbf0BdxdNo'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
    

In [ ]:
#import spotipy
#import csv
#from spotipy.oauth2 import SpotifyClientCredentials
#import pandas as pd

#auth_manager = SpotifyClientCredentials(client_id="68eea273ba0b4954a6237107625186fe",
#                                        client_secret = "18597e9724e945c8a64f86a2093aaea3")
#sp = spotipy.Spotify(auth_manager=auth_manager)